# Data Science Final Project

In this project, I aim to further the analysis for my arts and jobs paper. 
My main research question is: what drives growth, jobs or amenities? Do people follow jobs or amenities or both? 
Using regression analysis in the previous paper, I found that arts are a driver of growth. However, it is stronger in some places than others; for example more populated areas, and more educated areas. But what really is the underelying variables that would distinguish why arts are more influential in one places than in others? What variables would be more relevant in the jobs and amenities growth?

With over 50 control variables available, I would like to know which ones would help describe the changes better than only the main regression variables. 

In [1]:
%matplotlib inline
from collections import Counter
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans, DBSCAN
import seaborn as sns

In [2]:
df = pd.read_csv("sample_data")

In [10]:
var = pd.read_csv("control_variables.csv")

In [11]:
var.describe()

,Unnamed: 0,zip,MedAge2k,pctba2k,pctba90,racediversity2k,racediversity90,medrent2k,medrent90,medhhinc2k,...,Pop1990,RentMed,Democ92,NonWhite_90,CollProfLv90,CrimeRate1999county,ARTGOSLG98,Lat,Long,TotalWages
count,41263.000000,41263.000000,31854.000000,31761.000000,31562.000000,31771.000000,31576.000000,31854.000000,31727.000000,31709.000000,...,4.103500e+04,41035.000000,41035.000000,28563.000000,28446.000000,37183.000000,17739.000000,41263.000000,41263.000000,2.849500e+04
mean,20632.000000,49502.720209,37.714397,18.111714,14.824691,0.219786,0.173766,491.538488,371.928421,39625.645936,...,4.274453e+05,389.854003,41.387038,1690.752092,0.148797,3705.341930,-0.207305,38.503834,-90.838332,1.925986e+08
std,11911.746415,27937.923787,5.865451,13.742643,11.391290,0.211137,0.189796,228.829242,166.001317,16242.013483,...,1.123134e+06,127.489911,11.257693,5180.415557,0.117980,2158.999799,1.121321,5.425484,16.325046,2.873816e+08
min,1.000000,501.000000,0.000000,0.000000,0.000000,0.000000,-0.012346,0.000000,0.000000,2499.000000,...,1.070000e+02,140.000000,0.000000,0.000000,0.000000,28.000000,-4.560932,-14.270000,-176.670000,0.000000e+00
25%,10316.500000,25908.500000,35.000000,9.266409,7.607174,0.047652,0.023933,367.000000,265.000000,29777.000000,...,2.587000e+04,295.000000,34.500000,8.000000,0.074174,2145.000000,-0.896614,35.010000,-97.220000,1.900569e+07
50%,20632.000000,48812.000000,37.700000,14.085630,11.314077,0.134986,0.086027,458.000000,334.000000,36250.000000,...,8.112900e+04,361.000000,40.400000,96.000000,0.113861,3244.000000,-0.247625,39.110000,-87.880000,5.530192e+07
75%,30947.500000,72844.000000,40.500000,22.415940,17.833369,0.370209,0.289392,590.000000,445.000000,45744.000000,...,3.763960e+05,457.000000,47.100000,953.000000,0.182794,5002.000000,0.402646,41.870000,-80.020000,2.495607e+08
max,41263.000000,99950.000000,85.300000,100.000000,100.000000,0.842697,0.812095,2001.000000,1001.000000,200001.000000,...,8.863164e+06,926.000000,84.600000,102388.000000,1.000000,20965.000000,4.343458,71.280000,171.180000,2.132870e+09


We can list the control variables with the code below. And some variables of higher interest are listed on the table. 

In [12]:
list(var.columns.values)

['Unnamed: 0',
 'zip',
 'MedAge2k',
 'pctba2k',
 'pctba90',
 'racediversity2k',
 'racediversity90',
 'medrent2k',
 'medrent90',
 'medhhinc2k',
 'medhhinc90',
 'meantrav2k',
 'meantrav90',
 'medlengthres2k',
 'meanlengthres90',
 'pctsamehouse2k',
 'pctsamehouse90',
 'pctfemale2k',
 'pctfemale90',
 'pctmarried2k',
 'pctmarried90',
 'pctchild2k',
 'pctchild90',
 'pctunemp2k',
 'pctunemp90',
 'diversityfb2k',
 'diversityanc90',
 'pctrental2k',
 'pctrental90',
 'pctpov2k',
 'pctpov90',
 'pctbike2k',
 'pctbike90',
 'pctdrove2k',
 'pctdrove90',
 'pctwalk2k',
 'pctwalk90',
 'density2000',
 'density1990',
 'housingdensity2000',
 'housingdensity1990',
 'Pop2000',
 'Pop1990',
 'RentMed',
 'Democ92',
 'NonWhite_90',
 'CollProfLv90',
 'CrimeRate1999county',
 'ARTGOSLG98',
 'ZipCodeType',
 'City',
 'State',
 'LocationType',
 'Lat',
 'Long',
 'Location',
 'TotalWages']


| Variable | Description  |
|------|------|
| Pop2000 | Population in 2000 |
| CrimeRate1999county | Crime Rate in 1999 | 
| Democ92 | Percent Democrats | 
|  pctba2k  | Percent Graduated Bachelor Degree in 2000|
| pctrental2k | Percent Rentals in 2000 | 
| pctmarried2k | Percent Married Households in 2000 | 
| pctwalk2k | Percent of people who walk in 2000 | 


Besides these variables, I also have a number of establishments, and number of industries in each zip code. I could also get a list of individual industries present in each zip code from the raw data. And the Scenes data (my advisor's research), which coded each zip code on 15 scenes dimensions such as authenticity, corporativeness, etc. 

It would be specially interesting to do an analysis with the Scenes dimensions to see how the arts and jobs behave according to them. (And my advisor would like it too). 

I would like to do principal component analysis on the data, to see how the scenes dimensions group together for the "optimum" level of jobs by arts growth.

I also would like to try autoregression on the variables on themselves over time. 